# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')


InterfaceError: ('IM002', '[IM002] [unixODBC][Driver Manager]Data source name not found and no default driver specified (0) (SQLDriverConnect)')

In [3]:
students = pd.read_sql("SELECT * FROM Students;", conn)
courses = pd.read_sql("SELECT * FROM Courses;", conn)
classrooms = pd.read_sql("SELECT * FROM Classrooms;", conn)
teachers = pd.read_sql("SELECT * FROM Teachers;", conn)
departments = pd.read_sql("SELECT * FROM Departments;", conn)

NameError: name 'conn' is not defined

In [4]:
classroom_courses = pd.merge(classrooms, courses, on='CourseID', how='inner')
class_courses_dep = pd.merge(classroom_courses,departments, on='DepartmentID', how='inner')
class_courses_dep_stud = pd.merge(class_courses_dep,students, on='StudentID', how='inner')

# Questions
 
 
## Q1. Count the number of students on each department

In [5]:

students_departments = class_courses_dep.groupby(["DepartmentID","DepartmentName"]).agg({"StudentID": ['nunique']}).rename(columns = {"StudentID":"Student Number"})
print(students_departments)

                            Student Number
                                   nunique
DepartmentID DepartmentName               
1            English                    84
2            Science                   200
3            Arts                      137
4            Sport                      81


## Q2. How many students have each course of the English department and the total number of students in the department?

In [6]:
students_english_dep = class_courses_dep.query("DepartmentID==1")
students_Eng_Courses = students_english_dep.groupby(["CourseID","CourseName"]).agg({"StudentID": ['count']}).rename(columns = {"StudentID":"Student Number"})
print(students_Eng_Courses)

total_stud_eng_dep = students_english_dep.drop_duplicates(subset='StudentID', keep='first')['StudentID'].count()
print("\nTotal number of students in English department is", total_stud_eng_dep)

                              Student Number
                                       count
CourseID CourseName                         
1        English Begginers                32
2        Advanced English                 24
3        Proffesional English             38

Total number of students in English department is 84


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [7]:
students_english_dep = class_courses_dep.query("DepartmentID==2")
students_num_Sci_Courses = students_english_dep.groupby(["CourseID","CourseName"]).agg({"StudentID": ['count']})
students_num_Sci_Courses.loc[:,'Type'] = "Big"
students_num_Sci_Courses.loc[students_num_Sci_Courses.iloc[:,0]>21,'Type'] = "Small"
big_small= students_num_Sci_Courses.groupby(["Type"]).agg({"Type": ['count']})
print(big_small)

       Type
      count
Type       
Big       6
Small    10


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [8]:
print(students.groupby(["Gender"]).agg({"StudentID": ['count']}))
print("The feminist are WRONG, There are more women than man in the college")

       StudentID
           count
Gender          
F            165
M            115
The feminist are WRONG, There are more women than man in the college


## Q5. For which courses the percentage of male/female students is over 70%?

In [9]:
female = class_courses_dep_stud[(class_courses_dep_stud.Gender == "F")]
male = class_courses_dep_stud[(class_courses_dep_stud.Gender == "M")]
Total = class_courses_dep_stud.groupby(['CourseName']).agg(Total=("StudentID", "count"))

male = male.groupby(['CourseName']).agg(males=("StudentID", "count"))
female = female.groupby(['CourseName']).agg(females=("StudentID", "count"))
male_female = pd.merge(male, female,  on="CourseName" ,how = 'inner')
male_female = pd.merge(male_female, Total,  on="CourseName" ,how = 'inner')

male_female['perc_m'] = male_female['males'] / male_female ['Total'] > 0.7
male_female['perc_f'] = male_female['females'] / male_female ['Total'] > 0.7
male_female = male_female[(male_female.perc_f == True) | (male_female.perc_m == True) ]
print(male_female.iloc[:,0:3])

            males  females  Total
CourseName                       
Sculpture       7       17     24
Tenis           5       13     18


## Q6. For each department, how many students passed with a grades over 80?

In [12]:
depart_stud_80 = class_courses_dep_stud[(class_courses_dep_stud.degree >= 80)]
depart_stud_80 = depart_stud_80.groupby(['DepartmentName']).agg(Students_number=("StudentID", "count"))
print(depart_stud_80)

                Students_number
DepartmentName                 
Arts                         68
English                      43
Science                     171
Sport                        41


## Q7. For each department, how many students passed with a grades under 60?

In [13]:
depart_stud_60 = class_courses_dep_stud[(class_courses_dep_stud.degree <= 60)]
depart_stud_60 = depart_stud_60.groupby(['DepartmentName']).agg(Students_number=("StudentID", "count"))
print(depart_stud_60)

                Students_number
DepartmentName                 
Arts                          3
English                       2
Science                       7
Sport                         1


## Q8. Rate the teachers by their average student's grades (in descending order).

## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).